submit a shell command as a batch job.
See https://github.com/sciserver/SciScript-Python/blob/Feature_Jobs/py3/SciServer/Jobs.py

In [28]:
from SciServer import Jobs,Authentication,Config
import json
import requests
import ipywidgets as widgets
from os import listdir, walk
from os.path import isfile, isdir, join
import time
from datetime import datetime

## Functions to submit the job

In [29]:
def pathForUserVolume(uv):
    return '{0}/{1}/{2}'.format(uv['rootVolumeName'],uv['owner'],uv['name'])

In [30]:
# the following code replaces similar function in SciServer.Jobs
# That verison does not yet support writing to a data volume
def submitShellCommandJob(shellCommand, dockerComputeDomain = None, dockerImageName = None, userVolumes = None, dataVolumes = None, resultsFolderPath = "", jobAlias = ""):
    """
    Submits a shell command for execution (as an asynchronous job) inside a Docker compute domain.
    :param shellCommand: shell command (string) defined by the user.
    :param dockerComputeDomain: object (dictionary) that defines a Docker compute domain. A list of these kind of objects available to the user is returned by the function Jobs.getDockerComputeDomains().
    :param dockerImageName: name (string) of the Docker image for executing the notebook. E.g.,  dockerImageName="Python (astro)". An array of available Docker images is defined as the 'images' property in the dockerComputeDomain object.
    :param userVolumes: a list with the names of user volumes (with optional write permissions) that will be mounted to the docker Image.
           E.g., userVolumes = [{'name':'persistent', 'needsWriteAccess':False},{'name':'scratch', , 'needsWriteAccess':True}]
           A list of available user volumes can be found as the 'userVolumes' property in the dockerComputeDomain object. If userVolumes=None, then all available user volumes are mounted, with 'needsWriteAccess' = True if the user has Write permissions on the volume.
    :param dataVolumes: a list with the names of data volumes that will be mounted to the docker Image.
           E.g., dataVolumes=[{"name":"SDSS_DAS"}, {"name":"Recount"}].
           A list of available data volumes can be found as the 'volumes' property in the dockerComputeDomain object. If dataVolumes=None, then all available data volumes are mounted.
    :param resultsFolderPath: full path to results folder (string) where the shell command is executed. E.g.: /home/idies/workspace/rootVolume/username/userVolume/jobsFolder. If not set, then a default folder will be set automatically.
    :param jobAlias: alias (string) of job, defined by the user.
    :return: the job ID (int)
    :raises: Throws an exception if the HTTP request to the Authentication URL returns an error. Throws an exception if the HTTP request to the JOBM API returns an error, or if the volumes defined by the user are not available in the Docker compute domain.
    :example: dockerComputeDomain = Jobs.getDockerComputeDomains()[0]; job = Jobs.submitShellCommandJob('pwd', dockerComputeDomain, 'Python (astro)', [{'name':'persistent'},{'name':'scratch', 'needsWriteAccess':True}], [{'name':'SDSS_DAS'}], 'myNewJob')
    .. seealso:: Jobs.submitNotebookJob, Jobs.getJobStatus, Jobs.getDockerComputeDomains, Jobs.cancelJob
    """

    token = Authentication.login(UserName="------", Password="------")
    if token is not None and token != "":

        if Config.isSciServerComputeEnvironment():
            taskName = "Compute.SciScript-Python.Jobs.submitShellCommandJob"
        else:
            taskName = "SciScript-Python.Jobs.submitShellCommandJob"

        if dockerComputeDomain is None:
            dockerComputeDomains = getDockerComputeDomains();
            if dockerComputeDomains .__len__() > 0:
                dockerComputeDomain = dockerComputeDomains[0];
            else:
                raise Exception("There are no dockerComputeDomains available for the user.");

        if dockerImageName is None:
            images = dockerComputeDomain.get('images');
            if images.__len__() > 0:
                dockerImageName = images[0].get('name')
            else:
                raise Exception("dockerComputeDomain has no docker images available for the user.");

        uVols = [];
        for uVol in userVolumes:
            found = False;
            for vol in dockerComputeDomain.get('userVolumes'):
                if vol.get('name') == uVol.get('name'):
                    found = True;
                    if (uVol.get('needsWriteAccess')):
                        if uVol.get('needsWriteAccess') == True and 'write' in vol.get('allowedActions'):
                            uVols.append({'userVolumeId': vol.get('id'), 'needsWriteAccess': True});
                        else:
                            uVols.append({'userVolumeId': vol.get('id'), 'needsWriteAccess': False});
                    else:
                        if 'write' in vol.get('allowedActions'):
                            uVols.append({'userVolumeId': vol.get('id'), 'needsWriteAccess': True});
                        else:
                            uVols.append({'userVolumeId': vol.get('id'), 'needsWriteAccess': False});

            if not found:
                raise Exception("User volume '" + uVol.get('name') + "' not found within Compute domain")

        datVols = [];
        for dVol in dataVolumes:
            found = False;
            for vol in dockerComputeDomain.get('volumes'):
                name=dVol.get('name')
                print(name)
                if name == vol.get('name'):
                    found = True
                    if (vol.get('needsWriteAccess')):
                        print(vol.get('needsWriteAccess'))
                        print(vol.get('writable'))
                        if vol.get('needsWriteAccess') == True and vol.get('writable') == True:
                            datVols.append({'name': name, 'writable': True});
                        else:
                            datVols.append({'name': name, 'writable': False});
                    else:
                        if vol.get('writable'):
                            datVols.append({'name': name, 'writable': True});
                        else:
                            datVols.append({'name': name, 'writable': False});
                    found = True;

            if not found:
                raise Exception("Data volume '" + dVol.get('name') + "' not found within Compute domain")


        dockerComputeEndpoint = dockerComputeDomain.get('apiEndpoint');

        dockerJobModel = {
            "command": shellCommand,
            "submitterDID": jobAlias,
            "dockerComputeEndpoint": dockerComputeEndpoint,
            "dockerImageName": dockerImageName,
            "volumeContainers": datVols,
            "userVolumes": uVols,
            "resultsFolderURI": resultsFolderPath
        }
        data = json.dumps(dockerJobModel).encode()
        url = Config.RacmApiURL + "/jobm/rest/jobs/docker?TaskName="+taskName;
        headers = {'X-Auth-Token': token, "Content-Type": "application/json"}
        res = requests.post(url, data=data, headers=headers, stream=True)

        if res.status_code != 200:
            raise Exception("Error when submitting a job to the JOBM API.\nHttp Response from JOBM API returned status code " + str(res.status_code) + ":\n" + res.content.decode());
        else:
            return (json.loads(res.content.decode())).get('id')
    else:
        raise Exception("User token is not defined. First log into SciServer.")

### Sciserver User Config

In [31]:
#Your sciserver username and password
USERNAME='------'
PASSWORD='------'

# define the required job environment
DOMAIN='COVID-19 Jobs'  # change with name of new compute domain
IMAGE='SciServer Essentials'

# define lists of user and data volumes that should be mounted
# Add those that you 
USERVOLUMES=['Storage/'+USERNAME+'/persistent','Temporary/'+USERNAME+'/scratch']
DATAVOLUMES=['COVID-19']

RESULTSFOLDERPATH = "/home/idies/workspace/covid19/code/ncov/interactive_notebooks/jobs"
JOBALIAS = "covnorm-even_strand"


In [32]:
domains=Jobs.getDockerComputeDomains()
domain=None
image=None
volumes=[]
userVolumes=[]
dataVolumes=[]
for d in domains:
    if d['name'] == DOMAIN:
        domain=d
        for im in d['images']:
            if im['name'] == IMAGE:
                image = im
        for v in d['volumes']:
            if v['name'] in DATAVOLUMES:
                dataVolumes.append({"name":v['name'],'needsWriteAccess':True})
        for uv in d['userVolumes']:
            path=pathForUserVolume(uv)
            if path in USERVOLUMES:
                userVolumes.append({'name':uv['name'],'rootVolumeName':uv['rootVolumeName']
                                    ,'owner':uv['owner'],'needsWriteAccess':True})
        break

## Module 4: Draft Consensus 

Insert any command line arguements needed for scipt.  If more than 1 arguement is needed, you will have to modify the CMD variable below with extra arguements.  My example is providing a path where you want analysis results stored.

Nanopolish script...

In [33]:
script='bash -x /home/idies/workspace/Temporary/'+USERNAME+'/scratch/CoverageNormalization/normalization.sh'

Run Directory to process...

In [34]:
base_dir='/home/idies/workspace/covid19/sequencing_runs'

Find samples for the analysis...

In [40]:
def module_3_get_files(input_dir):

    files=[]

    for dirName, subdirList, fileList in walk(output_dir):
        for i, d in enumerate(subdirList):
            if(isdir(dirName+'/'+d+'/3-normalization')):
                run_path = '%s/%s/artic-pipeline/2-length-filter/' % (base_dir, d)
                for file in listdir(run_path):
                    if file.endswith(".fastq"):
                        files.append(join(run_path,file))
    return files

In [41]:
output_dir='/home/idies/workspace/Temporary/'+USERNAME+'/scratch/CoverageNormalization'
module_3_input_files=module_3_get_files(output_dir)
if len(module_3_input_files)==0:
    print("Input Fastq files for Module 3 not found!")
    

In [44]:
len(module_3_input_files)

156

Create command: merge bash script with parameters and input files for analysis...

In [45]:
commands=[]
for i in range(len(module_3_input_files)):
    module_3_job_no=i
    command = script + " " + module_3_input_files[i] " " + output_dir + " " + str(module_3_job_no)
    commands.append(command)


In [46]:
commands[0]

'bash -x /home/idies/workspace/Temporary/'+USERNAME+'/scratch/CoverageNormalization/normalization.sh /home/idies/workspace/covid19/sequencing_runs/20200405_0422_GA30000_FAN30842_00fb1614/artic-pipeline/2-length-filter/NTC_NB12.fastq 0'

In [47]:
len(commands)

156

## Run the analysis

### submit All Batches

In [63]:
def submit_batch(commands):
    jobs=[]
    for command in commands:
        job=Jobs.submitShellCommandJob(shellCommand=command
                                    , dockerComputeDomain = domain
                                    , dockerImageName = IMAGE
                                    , userVolumes = userVolumes, dataVolumes=dataVolumes
                                    , resultsFolderPath = "/home/idies/workspace/Temporary/'+USERNAME+'/scratch/jobs"
                                    , jobAlias = JOBALIAS)
        jobs.append(job)
        time.sleep(30)
    return jobs
    

In [64]:
jobs=submit_batch(commands)

KeyboardInterrupt: 

### submit Single

In [58]:
command=commands[0]
job=Jobs.submitShellCommandJob(shellCommand=command
                            , dockerComputeDomain = domain
                            , dockerImageName = IMAGE
                            , userVolumes = userVolumes, dataVolumes=dataVolumes
                            , resultsFolderPath = "/home/idies/workspace/Temporary/'+USERNAME+'/scratch/jobs"
                            , jobAlias = JOBALIAS)
print(job)

64858


In [62]:
print(Jobs.getJobStatus(64858))

{'status': 32, 'statusMeaning': 'SUCCESS', 'jobId': 64858}


In [174]:
#cancel single job
Jobs.cancelJob(64479)

In [217]:
#cancel batch jobs
#for job in jobs:
#    Jobs.cancelJob(job)